In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sc
import shap
import lime
import sklearn 
import warnings
import xgboost
import multiprocessing as mp
import time
import pickle 
import interpret
import collections
import math
import random

from sklearn.ensemble import AdaBoostRegressor
%matplotlib inline
import os, sys
#import statsmodels.api as sm
sys.path.append(os.path.abspath("../../../"))

pd.set_option('display.max_columns', None)
import eli5
from eli5.sklearn import PermutationImportance
import statsmodels.api as sm


The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
Using TensorFlow backend.


In [38]:
# Load Data
reopening_data = pd.read_csv('iped_reopen.csv')
college_cases = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/colleges/colleges.csv')
reopening_status = pd.read_csv('ReopeningStatusFinal.csv')
demographics = pd.read_csv('demographic_data_county.csv')
college_details = pd.read_csv('College Details.csv')
politics = pd.read_csv('politics_county.csv')
testing_plans = pd.read_csv('college_testing_plans.csv')
device_counts = pd.read_csv('college_cbg_device_counts.csv')

reopening_data.drop('Unnamed: 0',axis = 1,inplace = True)
college_cases = college_cases[~college_cases['ipeds_id'].isnull()]
college_cases['ipeds_id'] = college_cases['ipeds_id'].astype(int)
reopening_status.drop('Unnamed: 0',axis = 1 ,inplace = True)
reopening_data['pct_decrease'] = -(reopening_data['visits_by_day_2020'] -reopening_data['visits_by_day_2019'])/reopening_data['visits_by_day_2019']

college_data = pd.merge(reopening_data,college_cases,left_on = 'unitid',right_on = 'ipeds_id',how = 'left')
college_data['normalized_cases_college'] = college_data['cases']/college_data['enrtot']*1000
college_data = college_data[['unitid','instnm','enrtot','pct_decrease','normalized_cases_college','state_abbrev']]

demo_to_select = ['FIPS','State','ratio_population_enrolled-','Rural-urban_Continuum Code_2013','normalized_cases_prior_start','Median_Household_Income_2018','Unemployment_rate_2018']
demo_merge = demographics[demo_to_select]
percent_gop = politics[['combined_fips','per_gop']]

testing_plans_merge = testing_plans[['institution','testing','testing_type','test_recurring','test_freq','test_pool']]
device_counts.drop(['Unnamed: 0','inst_name'],axis = 1, inplace = True)


all_data = pd.merge(reopening_status,college_data,left_on = 'UnitID', right_on = 'unitid')
all_data.drop(['instnm','unitid'],axis = 1,inplace = True)
all_data = all_data[all_data['pct_decrease']<= 2]
all_data = all_data[all_data['pct_decrease']>= -2]
all_data = all_data.rename({"pct_decrease":'pct_decrease_oncampus'})

all_data = pd.merge(all_data,demo_merge,on = 'FIPS', how = 'left')
all_data = pd.merge(all_data, percent_gop, left_on = 'FIPS', right_on = 'combined_fips')
all_data = pd.merge(all_data,device_counts,left_on = 'UnitID',right_on = 'unitid', how = 'left' )
all_data = pd.merge(all_data,testing_plans_merge,left_on = 'Institution Name', right_on = 'institution',how = 'left')
all_data.to_csv('college_FA2020_details.csv')
